In [1]:
import theano
import theano.tensor as T
import numpy as np

In [140]:
class LSTM_cell(object):
    
    
    def __init__(self):
        self.random_state = np.random.RandomState(284783)
        self.output_dim = 10
        self.outer_output_dim = 1
        self.input_dim = self.outer_output_dim
        self.layer_id = "0"
        self.learning_rate = 1
        
        self.initialize_params()
        
       
        
        
    def initialize_params(self):
        U_input = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     (self.output_dim, self.input_dim))
            , dtype=theano.config.floatX)

        U_forget = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     (self.output_dim, self.input_dim))
            , dtype=theano.config.floatX)

        U_output = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     (self.output_dim, self.input_dim))
            , dtype=theano.config.floatX)

        W_input = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     (self.output_dim, self.output_dim))
            , dtype=theano.config.floatX)

        W_forget = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     (self.output_dim, self.output_dim))
            , dtype=theano.config.floatX)

        W_output = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     (self.output_dim, self.output_dim))
            , dtype=theano.config.floatX)

        U = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     (self.output_dim, self.input_dim))
            , dtype=theano.config.floatX)

        W = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     (self.output_dim, self.output_dim))
            , dtype=theano.config.floatX)

        bias_input = np.zeros(self.output_dim, dtype=theano.config.floatX)
        bias_forget = np.zeros(self.output_dim, dtype=theano.config.floatX)
        bias_output = np.zeros(self.output_dim, dtype=theano.config.floatX)
        bias = np.zeros(self.output_dim, dtype=theano.config.floatX)

        
        self.W = theano.shared(value=W, name="W" + self.layer_id, borrow="True")
        self.U = theano.shared(value=U, name="U" + self.layer_id, borrow="True")
        self.bias = theano.shared(value=bias, name="bias", borrow="True")

        self.W_input = theano.shared(value=W_input, name="W_input" + self.layer_id, borrow="True")
        self.U_input = theano.shared(value=U_input, name="U_input" + self.layer_id, borrow="True")
        self.bias_input = theano.shared(value=bias_input, name="bias_input" + self.layer_id, borrow="True")

        self.W_output = theano.shared(value=W_output, name="W_output" + self.layer_id, borrow="True")
        self.U_output = theano.shared(value=U_output, name="U_output" + self.layer_id, borrow="True")
        self.bias_output = theano.shared(value=bias_output, name="bias_output" + self.layer_id, borrow="True")

        self.W_forget = theano.shared(value=W_forget, name="W_forget" + self.layer_id, borrow="True")
        self.U_forget = theano.shared(value=U_forget, name="U_forget" + self.layer_id, borrow="True")
        self.bias_forget = theano.shared(value=bias_forget, name="bias_forget" + self.layer_id, borrow="True")
        self.W = theano.shared(value=W, name="W" + self.layer_id, borrow="True")
        self.U = theano.shared(value=U, name="U" + self.layer_id, borrow="True")
        self.bias = theano.shared(value=bias, name="bias", borrow="True")
        O_w = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                      np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                      (self.outer_output_dim,self.output_dim))
            , dtype=theano.config.floatX)

        O_bias = np.zeros(self.outer_output_dim, dtype=theano.config.floatX)

        self.O_w = theano.shared(value=O_w, name="O_w"+self.layer_id, borrow="True")
        self.O_bias = theano.shared(value=O_bias, name="O_bias"+self.layer_id, borrow="True")

        self.params = [self.U_input, self.U_forget, self.U_output, self.W_input, self.W_forget, self.W_output,
                       self.bias_input, self.bias_forget, self.bias_output, self.U, self.W, self.bias,
                       ]

        self.output_params = [self.O_w,self.O_bias]
    

    
    def build_model(self):
        def forward_step(x_t,prev_state,prev_content):
            input_gate = T.nnet.hard_sigmoid(T.dot(self.U_input,x_t) + T.dot(self.W_input,prev_state) + self.bias_input)
            forget_gate = T.nnet.hard_sigmoid(T.dot(self.U_forget,x_t) + T.dot(self.W_forget,prev_state) + self.bias_forget)
            output_gate = T.nnet.hard_sigmoid(T.dot(self.U_output,x_t) + T.dot(self.W_output,prev_state) + self.bias_output)

            stabialized_input = T.tanh(T.dot(self.U,x_t) + T.dot(self.W,prev_state) + self.bias)
            c = forget_gate * prev_content + input_gate * stabialized_input
            s = output_gate * T.tanh(c)

            o = T.nnet.hard_sigmoid(T.dot(self.O_w,s) + self.O_bias)

            return [o,s,c]
        
        params = self.params + self.output_params
        
        initial_hidden_state = T.matrix('H')
        y = T.matrix('Y')
        
        [output,states,mem_content],updates = theano.scan(forward_step,n_steps=10,
                                                          outputs_info= [
                #dict(initial=np.asarray(self.random_state.normal(-np.sqrt(6.0 / (self.outer_output_dim)),np.sqrt(6.0 / (self.outer_output_dim)),
                                    # (self.outer_output_dim))
            #, dtype=theano.config.floatX)), 
                dict(initial=y),
dict(initial=np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.output_dim)),
                                     np.sqrt(6.0 / (self.output_dim)),
                                     (self.output_dim,1))
            , dtype=theano.config.floatX)), 
dict(initial=np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.output_dim)),
                                     np.sqrt(6.0 / (self.output_dim)),
                                     (self.output_dim,1))
            , dtype=theano.config.floatX))])
                                                      
        self.forward = theano.function([],[output])            
        #cost = T.sum(
        #        ((T.ones(self.output_dim,dtype=theano.config.floatX)) - 
        #    (output[-1])) ** 2
        #)
        
        cost = 1 - 2*T.round(T.max(abs(y - output)))
        grads = theano.grad(cost,params) # [2*cost - 1 for p in params] #
        updates = [ (p,p + self.learning_rate*(cost*p) + np.random.rand()) for p,g in zip(params,grads)]
        self.training_step = theano.function([y],[T.max(grads[0]),cost,output,y], updates=updates)
       
        
            
    
    


lstm_cell = LSTM_cell()
lstm_cell.build_model()
    

MissingInputError: An input of the graph, used to compute DimShuffle{x,0,1}(Y), was not provided and not given a value.Use the Theano flag exception_verbosity='high',for more information on this error.

Backtrace when the variable is created:
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 228, in dispatch_shell
    handler(stream, idents, msg)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 391, in execute_request
    user_expressions, allow_stdin)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/ipkernel.py", line 199, in do_execute
    shell.run_cell(code, store_history=store_history, silent=silent)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2723, in run_cell
    interactivity=interactivity, compiler=compiler, result=re

In [138]:
a = lstm_cell.forward_step()
print(a)

AttributeError: 'LSTM_cell' object has no attribute 'forward_step'

In [129]:
lstm_cell = LSTM_cell()
lstm_cell.build_model()

for i in np.arange(1000):
    H = np.random.binomial(1, 0, size=10)
    Y = np.array([ 1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.], dtype=theano.config.floatX) #np.asarray([h for h in H],dtype=theano.config.floatX) 
    #print(Y)#[float(h) for h in H]
    #X = [0 for y in Y]
    [grad,cost,outputs,y] = lstm_cell.training_step(Y)
    if cost == 1:
        lstm_cell.learning_rate = lstm_cell.learning_rate / 10
    #print(lstm_cell.learning_rate)
    print(cost)
    print(outputs)
    print(y)



-1.0
[ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
-1.0
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
-1.0
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
-1.0
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
-1.0
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
-1.0
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
-1.0
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
-1.0
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
-1.0
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
-1.0
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
-1.0
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
-1.0
[ 1.  

In [117]:
lstm_cell.build_model()
print(np.argmax(outputs))

0


In [118]:
print(outputs)
print(np.argmax(outputs,axis=1))

[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]


ValueError: axis(=1) out of bounds